In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

In [7]:
# get data from AKShare
import akshare as ak
import pandas as pd

def get_data_from_akshare(symbol="QQQ", adjust="qfq"):
    data = ak.stock_us_daily(symbol=symbol, adjust=adjust).iloc[:,:7]
    # 把 date 作为日期索引，以符合 Backtrader 的要求
    data.index = pd.to_datetime(data['date'])
    return data

In [9]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    ## load daily data from AKShare
    stock_id = 'QQQ'
    start_date = datetime.datetime(2024, 1, 1)  # 回测开始时间
    end_date = datetime.datetime(2024, 12, 31)  # 回测结束时间
    stock_qfq_df = get_data_from_akshare(symbol=stock_id)
    ## load data
    data = bt.feeds.PandasData(dataname=stock_qfq_df, fromdate=start_date, 
                               todate=end_date, datetime='date', open='open', 
                               high='high',low='low', close='close', volume='volume')

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2024-01-02, Close, 399.74
2024-01-03, Close, 395.48
2024-01-04, Close, 393.43
2024-01-05, Close, 393.90
2024-01-08, Close, 402.10
2024-01-09, Close, 402.90
2024-01-10, Close, 405.65
2024-01-11, Close, 406.50
2024-01-12, Close, 406.71
2024-01-16, Close, 406.67
2024-01-17, Close, 404.36
2024-01-18, Close, 410.14
2024-01-19, Close, 418.33
2024-01-22, Close, 418.88
2024-01-23, Close, 420.63
2024-01-24, Close, 422.98
2024-01-25, Close, 423.50
2024-01-26, Close, 420.96
2024-01-29, Close, 425.30
2024-01-30, Close, 422.45
2024-01-31, Close, 414.12
2024-02-01, Close, 419.03
2024-02-02, Close, 426.16
2024-02-05, Close, 425.60
2024-02-06, Close, 424.74
2024-02-07, Close, 429.14
2024-02-08, Close, 429.94
2024-02-09, Close, 434.20
2024-02-12, Close, 432.49
2024-02-13, Close, 425.70
2024-02-14, Close, 430.37
2024-02-15, Close, 431.66
2024-02-16, Close, 427.72
2024-02-20, Close, 424.47
2024-02-21, Close, 422.76
2024-02-22, Close, 435.22
2024-02-23, Close, 433.93
20